# Setup

Tutorial for reddit scraping: https://www.geeksforgeeks.org/scraping-reddit-using-python/

In [ ]:
import praw
import pandas as pd
from praw.models import MoreComments
# from tqdm.notebook import tqdm
from tqdm import tqdm


# ids for scraping (from christians setup)
client_id = 'Ut5UgaAMOEWBELtYRWnw0g'
client_secret = '5xGs1w6mav5Ke685afpG28Q8nfusmg'
user_agent = 'polarity search'

# Scraping

First we initialize a read-only instance. A read-only instance can only scrape publicly available information and cannot upvote or otherwise interact like users can.

In [ ]:
# Read-only instance
reddit_read_only = praw.Reddit(client_id=client_id,         # your client id
                               client_secret=client_secret,      # your client secret
                               user_agent=user_agent)        # your user agent

## Getting comments on a specific post

This code scrapes over the comments of a specified post. It looks only at the top-level comments (none of the replies to comments).

In [ ]:
def scrape_post(url, n=100):
    '''given a url:
     - scrapes n number of top-level comments
     - comment author (username) and comment body ()
     - outputs a pandas dataframe'''

    # Creating a submission object
    submission = reddit_read_only.submission(url=url)
    
    # should get all top level comments on the post
    #if all_comments==True:
    #    submission.comments.replace_more(limit=None)

    post_authors = []
    post_comments = []

    # specifying how many times we should "load more comments"
    limit = round(n/20) # n/20 ensures we hit load more enough times to scrape the number of comments we want 
    submission.comments.replace_more(limit=limit, threshold=0)

    count = 0 # tracking how many comments have been scraped
    for comment in submission.comments: # iterates only over top level comments
        # stops counting when n amount of comments have been scraped
        if count == n:
            break

        if type(comment) == MoreComments:
            continue

        post_authors.append(comment.author)
        post_comments.append(comment.body)

        count += 1

    post_dict = {'author': post_authors, 'comment': post_comments}
    post_df = pd.DataFrame(post_dict)
    
    return post_df

### Examples of how scraping a post works

In [ ]:
# scraping a post without much content - takes <1 second
# there are 12 top level comments
df = scrape_post("https://www.reddit.com/r/MaraudersGame/comments/ylxsq4/marauders_be_like/")

print(df.shape)
df.head()

In [ ]:
# scraping a decently sized post (function default scrapes 100 comments) - takes ~10 seconds
df = scrape_post("https://www.reddit.com/r/politics/comments/1092xhl/the_american_public_no_longer_believes_the/")

print(df.shape)
df.head()

In [ ]:
# scraping 200 comments from a decently sized post (post has 3.8k comments) - takes ~15 seconds
df = scrape_post("https://www.reddit.com/r/politics/comments/1092xhl/the_american_public_no_longer_believes_the/", n=200)

print(df.shape)
df.head()

In [ ]:
# aidan: idk what below line was so i just commented it out
# praw.models.reddit.submission.Submission?

## Getting top month posts on specified subreddit
This code grabs the top 100 posts of the past month and saves various information on them into a dictionary

In [ ]:
def scrape_top_month(subreddit, ppsr=100):
    # specifying subreddit
    subreddit = reddit_read_only.subreddit(subreddit)

    # Specifying to look at top posts of the current month
    posts = subreddit.top("month", limit=ppsr)

    # Initializing dictionary to save post data to
    posts_dict = {"Title": [], "Post Text": [],
                  "ID": [], "Score": [],
                  "Total Comments": [], "Post URL": [], 'Post_author' : []
                  }

    # Loop for saving post details
    for post in posts:
        # print(post)
        # Title of each post
        posts_dict["Title"].append(post.title)

        # Text inside a post
        posts_dict["Post Text"].append(post.selftext)

        # Unique ID of each post
        posts_dict["ID"].append(post.id)

        # The score of a post
        posts_dict["Score"].append(post.score)

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(post.num_comments)

        # print(post.author)
        # Author of the post
        posts_dict['Post_author'].append(post.author)

        # URL of each post
        # print('https://www.reddit.com'+f'{post.permalink}')
        posts_dict["Post URL"].append('https://www.reddit.com'+f'{post.permalink}')
        
    return posts_dict

In [ ]:
dict_ = scrape_top_month('politics')

In [ ]:
# post samples
print(dict_['Title'][0])
print(dict_['Post Text'][0])
print(dict_['ID'][0])
print(dict_['Score'][0])
print(dict_['Total Comments'][0])
print(dict_['Post URL'][0])
print(len(dict_['Title']))
print(dict_['Post_author'][0])

In [ ]:
dict_ = scrape_top_month('politics', ppsr=150)

In [ ]:
# post samples
print(dict_['Title'][0])
print(dict_['Post Text'][0])
print(dict_['ID'][0])
print(dict_['Score'][0])
print(dict_['Total Comments'][0])
print(dict_['Post URL'][0])
print(len(dict_['Title']))
print(dict_['Post_author'])

## Getting comments on top monthly posts on multiple subreddits

In [ ]:
def scrape_multiple_save(subreddits, ppsr=100, all_comments=False):
    '''scrapes and saves subreddit comments to csv files
       Naming convention is: SUBREDDIT_POSTID.csv / SUBREDDIT_POSTID_INFO.txt'''
    
    
    if all_comments==False:
        print(f'Scraping {ppsr} posts per subreddit and ~100 comments per post')
    else:
        print(f'Scraping {ppsr} posts per subreddit and all comments per post')
    
    # looping through subreddits
    for subreddit in subreddits:
        print(f'Scraping r/{subreddit}...')
        
        # initialize dictionary for saving all comments and post info
        sub_dict = {'post_title': [],
                    'post_text': [],
                    'post_id': [],
                    'post_score': [],
                    'post_total_comments': [],
                    'post_url': [],
                    'comment_author': [],
                    'comment_text': [], 
                    'post_author' : []}
        
        posts_dict = scrape_top_month(subreddit, ppsr) # getting top of the month post info
        
        # looping through posts
        for idx, url in tqdm(enumerate(posts_dict['Post URL']),):
            
            # df for comments on the post
            comment_df = scrape_post(url, all_comments=all_comments)
            
            # looping through comments on post and appending all comment info to sub_dict
            for row_idx, row in comment_df.iterrows():
                sub_dict['post_title'].append(posts_dict['Title'][idx])
                sub_dict['post_text'].append(posts_dict['Post Text'][idx])
                sub_dict['post_id'].append(posts_dict['ID'][idx])
                sub_dict['post_score'].append(posts_dict['Score'][idx])
                sub_dict['post_total_comments'].append(posts_dict['Total Comments'][idx])
                sub_dict['post_url'].append(posts_dict['Post URL'][idx])
                sub_dict['comment_author'].append(row['author'])
                sub_dict['comment_text'].append(row['comment'])
                sub_dict['post_author'].append(posts_dict['Post_author'][idx])
            
        # changing sub_dict to pandas dataframe
        global sub_df
        sub_df = pd.DataFrame.from_dict(sub_dict)

        # saving to csv
        #sub_df.to_csv(f'../data/28feb/scrapes/{subreddit}.csv', index=False)
        
    print('Done!')
    return None

In [ ]:
scrape_multiple_save(['politics'])

In [ ]:
sub_df

In [ ]:
sub_df.to_csv(f'../data/28feb/scrapes/politics.csv', index=False)